# Расчёт влияния каналов

Данные из себя представляют выгрузку из системы сквозной аналитики Smaris за апрель 2025 г., где:
* `Person ID`	— уникальный идентификатор клиента
* `Дата создания` — Дата конверсии
* `Канал` — касания пользователя через запятую. Цифры в колонке "Канал" после знака ":" означают долю канала во всех касаниях пользователя по атрибуции "Линейное распределение" за период 180 дней.
* `Площадка` - расшифровка канала

Задача: Необходимо рассчитать влияние кажого канала на конверсии.

## Загрузка и обработка данных

In [1]:
import pandas as pd
from collections import defaultdict

In [2]:
df = pd.read_csv('/content/Smartis_OD_Export_7888_2025_05_14_22_29_37.csv', converters={'Person ID': str})
df

,Person ID,Дата создания,Канал,Площадка
0,946834193,2025-04-30 21:25:50,"Медийная реклама: 0.62, SEO: 0.19, Прямой траф...","cian.ru: 0.5, Google Поиск: 0.19, Прямой трафи..."
1,977655640,2025-04-30 20:43:21,Контекстная реклама: 1,Яндекс Директ: 1
2,,2025-04-30 18:17:37,Контекстная реклама: 1,Яндекс Директ: 1
3,977616728,2025-04-30 18:32:18,Базы недвижимости: 1,Авито (avito) База: 1
4,970733412,2025-04-30 19:03:02,Медийная реклама: 1,realty.ru: 1
...,...,...,...,...
774,979873460,2025-04-01 11:03:06,"Прямой трафик: 0.52, Медийная реклама: 0.23, S...","Прямой трафик // Общее: 0.52, Google Поиск: 0...."
775,961596403,2025-04-01 10:24:20,Реклама в соц.сетях: 1,vk.com реклама: 1
776,955090548,2025-04-01 10:40:31,Контекстная реклама: 1,Яндекс Директ: 1
777,961573036,2025-04-01 10:16:34,Медийная реклама: 1,gogethome: 1


In [9]:
def EDA_table(df):

    EDA = pd.DataFrame({
        'Columns': pd.Series(dtype='object'),
        'Types': pd.Series(dtype='object'),
        'Values': pd.Series(dtype='object'),
        'Len': pd.Series(dtype='int64'),
        'Non-null': pd.Series(dtype='int64'),
        'Uniques': pd.Series(dtype='int64'),
        'Uniques(no nulls)': pd.Series(dtype='int64'),
        'Missing(n)': pd.Series(dtype='int64'),
        'Missing(%)': pd.Series(dtype='float64')
    })


    for c in df.columns:
        eda = {
            'Columns': c,
            'Types': df[c].dtypes,
            'Values': [df[c].unique()],
            'Len': len(df[c]),
            'Non-null': len(df[c]) - df[c].isnull().sum(),
            'Uniques': len(df[c].unique()),
            'Uniques(no nulls)': df[c].nunique(),
            'Missing(n)': df[c].isnull().sum(),
            'Missing(%)': (df[c].isnull().sum() / len(df)).round(3) * 100,
            'Empty(n)': len(df[df[c] == ""])
        }


        EDA = pd.concat([EDA, pd.DataFrame([eda])], ignore_index=True)

    return EDA

EDA_table(df)

,Columns,Types,Values,Len,Non-null,Uniques,Uniques(no nulls),Missing(n),Missing(%),Empty(n)
0,Person ID,object,"[[946834193, 977655640, , 977616728, 970733412...",779,779,741,741,0,0.0,14.0
1,Дата создания,object,"[[2025-04-30 21:25:50, 2025-04-30 20:43:21, 20...",779,779,779,779,0,0.0,0.0
2,Канал,object,"[[Медийная реклама: 0.62, SEO: 0.19, Прямой тр...",779,779,300,300,0,0.0,0.0
3,Площадка,object,"[[cian.ru: 0.5, Google Поиск: 0.19, Прямой тра...",779,779,383,383,0,0.0,0.0


У нас 14 клиентов с пустым значением `Person ID`.
Заменим пустые значения на последовательность чисел, начиная с 1 и увеличивая на единицу.

**Шаг 1. Найдем пустые ячейки**

Пустыми считаем: NaN, None, "" (пустая строка).

In [11]:
# Для столбца 'A' находим пустые ячейки
mask = df['Person ID'].isna() | (df['Person ID'].astype(str).str.strip() == "")

`mask` будет содержать True для пустых ячеек.

**Шаг 2. Заменяем их на последовательность 1, 2, 3, ...**

In [14]:
# Находим индексы пустых ячеек
empty_indices = df.index[mask]

# Заменяем их на числа от 1 до len(empty_indices)
df.loc[empty_indices, 'Person ID'] = range(1, len(empty_indices) + 1)

In [15]:
df

,Person ID,Дата создания,Канал,Площадка
0,946834193,2025-04-30 21:25:50,"Медийная реклама: 0.62, SEO: 0.19, Прямой траф...","cian.ru: 0.5, Google Поиск: 0.19, Прямой трафи..."
1,977655640,2025-04-30 20:43:21,Контекстная реклама: 1,Яндекс Директ: 1
2,1,2025-04-30 18:17:37,Контекстная реклама: 1,Яндекс Директ: 1
3,977616728,2025-04-30 18:32:18,Базы недвижимости: 1,Авито (avito) База: 1
4,970733412,2025-04-30 19:03:02,Медийная реклама: 1,realty.ru: 1
...,...,...,...,...
774,979873460,2025-04-01 11:03:06,"Прямой трафик: 0.52, Медийная реклама: 0.23, S...","Прямой трафик // Общее: 0.52, Google Поиск: 0...."
775,961596403,2025-04-01 10:24:20,Реклама в соц.сетях: 1,vk.com реклама: 1
776,955090548,2025-04-01 10:40:31,Контекстная реклама: 1,Яндекс Директ: 1
777,961573036,2025-04-01 10:16:34,Медийная реклама: 1,gogethome: 1


## Рассчёт влияния каналов

Для расчета влияния каждого канала на основе предоставленных данных выполним следующие шаги:

**Шаг 1. Разбор данных**

* Для каждой строки в колонке "Канал" извлечем названия каналов и их доли (например, для строки `"Медийная реклама: 0.62, SEO: 0.19, Прямой трафик: 0.19"` каналы и доли будут: `Медийная реклама` — 0.62, `SEO` — 0.19, `Прямой трафик` — 0.19).

**Шаг 2. Агрегация данных:**

* Просуммируем доли каждого канала по всем пользователям. Это даст общий "вес" каждого канала.

**Шаг 3. Расчет влияния:**

* Разделим общий вес каждого канала на сумму всех весов, чтобы получить процентное влияние каждого канала.

In [33]:
# Словарь для хранения сумм долей каналов
channel_weights = defaultdict(float)

# Обработка каждой строки в колонке "Канал"
for channel_str in df['Канал']:
    if pd.isna(channel_str):
        continue
    # Разбиваем строку на части по запятым
    parts = [part.strip() for part in channel_str.split(',')]
    for part in parts:
      if ':' not in part:
          continue
      channel, weight_str = part.split(':')
      channel = channel.strip()
      try:
          weight = float(weight_str.strip())
      except ValueError:
          continue
      channel_weights[channel] += weight

# Сумма всех весов
total_weight = sum(channel_weights.values())

# Расчет влияния в процентах
channel_influence = {
    channel: (weight / total_weight) * 100
    for channel, weight in channel_weights.items()
}

# Сортируем каналы по убыванию влияния
sorted_influence = sorted(
    channel_influence.items(),
    key=lambda x: x[1],
    reverse=True
)

# Вывод результатов
print("Влияние каналов (в %):")
for channel, influence in sorted_influence:
    print(f"{channel}: {influence:.2f}%")

Влияние каналов (в %):
Медийная реклама: 33.35%
Контекстная реклама: 20.74%
Базы недвижимости: 18.34%
Прямой трафик: 11.05%
SEO: 4.91%
Реклама в соц.сетях: 3.92%
Карты: 3.53%
Программатик: 1.66%
Брокеры/Агенты: 0.89%
Реферальный трафик: 0.37%
Другая реклама: 0.37%
Наружная реклама: 0.35%
CRM-маркетинг: 0.27%
Соц.сети органика: 0.24%
Рекомендации: 0.01%


## Результат

На основе анализа данных из файла, влияние каналов распределено следующим образом:

Влияние каналов (в %):
* Медийная реклама: 33.35%
* Контекстная реклама: 20.74%
* Базы недвижимости: 18.34%
* Прямой трафик: 11.05%
* SEO: 4.91%
* Реклама в соц.сетях: 3.92%
* Карты: 3.53%
* Программатик: 1.66%
* Остальные: <1%
